In [1]:
import sys
!{sys.executable} -m pip install google-cloud-bigquery

In [2]:
!pip install prettytable

In [3]:
!pip install db-dtypes

In [4]:
!pip install pandas-gbq

In [5]:
from google.cloud import bigquery


# Create a BigQuery client
client = bigquery.Client(project="degroup11")

In [6]:
from prettytable import PrettyTable
import pandas as pd
from prettytable import PrettyTable

In [7]:
QUERY = ('SELECT ID, has_spouse, gross_salary, alimony_amount, gross_salary - 12 * alimony_amount AS spendable_income FROM `degroup11.group11dataset.individuals` LIMIT 100')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a PrettyTable
table = PrettyTable(columns)

# Add rows to the table
for row in rows:
    table.add_row([row[column] for column in columns])

# Print the table
print(table)

+-----+------------+--------------+----------------+------------------+
|  ID | has_spouse | gross_salary | alimony_amount | spendable_income |
+-----+------------+--------------+----------------+------------------+
| 106 |    True    |    75526     |       0        |      75526       |
| 153 |   False    |    134600    |      1200      |      120200      |
| 211 |    True    |    144798    |       0        |      144798      |
| 354 |    True    |    150580    |       0        |      150580      |
| 363 |   False    |    186763    |       0        |      186763      |
| 469 |    True    |    59116     |       0        |      59116       |
| 520 |    True    |    189883    |      3697      |      145519      |
| 539 |    True    |    157499    |       0        |      157499      |
| 594 |    True    |    171667    |      1858      |      149371      |
|  9  |   False    |    168304    |       0        |      168304      |
|  30 |    True    |    75892     |       0        |      75892 

In [8]:
QUERY = ('''
    SELECT 
    ind.gross_salary,
    ind.alimony_amount,
    ind.has_spouse,
    sp.gross_salary AS sp_gross_salary,
    sp.alimony_amount AS sp_alimony_amount,
    CAST(
        IF(ind.has_spouse = FALSE, 
            ind.gross_salary - 12 * ind.alimony_amount, 
            IF(ind.gross_salary > sp.gross_salary, 
            ind.gross_salary - 12 * ind.alimony_amount + 1/3 * (sp.gross_salary - 12 * sp.alimony_amount),
            sp.gross_salary - 12 * sp.alimony_amount + 1/3 * (ind.gross_salary - 12 * ind.alimony_amount))
        ) AS INT64
    ) AS house_spendable_income
FROM 
    degroup11.group11dataset.individuals ind
LEFT JOIN 
    degroup11.group11dataset.spouse sp
ON 
    ind.spouse_ID = sp.spouse_ID
    ''')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a PrettyTable
table = PrettyTable(columns)

# Add rows to the table
for row in rows:
    table.add_row([row[column] for column in columns])

# Print the table
print(table[:10])

+--------------+----------------+------------+-----------------+-------------------+------------------------+
| gross_salary | alimony_amount | has_spouse | sp_gross_salary | sp_alimony_amount | house_spendable_income |
+--------------+----------------+------------+-----------------+-------------------+------------------------+
|    75526     |       0        |    True    |      59122      |         0         |         95233          |
|    134600    |      1200      |   False    |      101412     |         0         |         120200         |
|    144798    |       0        |    True    |      149733     |         0         |         197999         |
|    150580    |       0        |    True    |      47582      |         0         |         166441         |
|    186763    |       0        |   False    |      181201     |         0         |         186763         |
|    59116     |       0        |    True    |      38986      |         0         |         72111          |
|    18988

In [9]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client()

QUERY = ('''
    SELECT 
    ind.gross_salary,
    ind.alimony_amount,
    ind.has_spouse,
    sp.gross_salary AS sp_gross_salary,
    sp.alimony_amount AS sp_alimony_amount,
    CAST(
        IF(ind.has_spouse = FALSE, 
            ind.gross_salary - 12 * ind.alimony_amount, 
            IF(ind.gross_salary > sp.gross_salary, 
            ind.gross_salary - 12 * ind.alimony_amount + 1/3 * (sp.gross_salary - 12 * sp.alimony_amount),
            sp.gross_salary - 12 * sp.alimony_amount + 1/3 * (ind.gross_salary - 12 * ind.alimony_amount))
        ) AS INT64
    ) AS house_spendable_income
FROM 
    degroup11.group11dataset.individuals ind
LEFT JOIN 
    degroup11.group11dataset.spouse sp
ON 
    ind.spouse_ID = sp.spouse_ID
''')

query_job = client.query(QUERY)  # API request

# Use list comprehension to convert rows to a list
rows = [row for row in query_job.result()]

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a DataFrame manually
result_df = pd.DataFrame(data=[list(row.values()) for row in rows], columns=columns)

# Now you can access and manipulate the data using Pandas functions
result_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income
0,75526,0,True,59122,0,95233
1,134600,1200,False,101412,0,120200
2,144798,0,True,149733,0,197999
3,150580,0,True,47582,0,166441
4,186763,0,False,181201,0,186763
...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811
996,111108,0,False,38953,0,111108
997,51688,0,True,47980,0,67681
998,193117,0,True,183747,0,254366


In [10]:
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client()

QUERY = ('''
SELECT 
    *
FROM 
    degroup11.group11dataset.toetsinkomen
''')

query_job = client.query(QUERY)  # API request

# Use list comprehension to convert rows to a list
rows = [row for row in query_job.result()]

# Get column names
columns = [field.name for field in query_job.result().schema]

# Create a DataFrame manually
rente_df = pd.DataFrame(data=[list(row.values()) for row in rows], columns=columns)

# Now you can access and manipulate the data using Pandas functions
rente_df

,Toetsinkomen,Toetsrente
0,0,1.75
1,19500,1.75
2,20000,1.85
3,20050,1.95
4,21000,2.05
5,21500,2.10
6,22000,2.20
7,22500,2.35
8,23000,2.45
9,23500,2.50


In [11]:
result_df['Toetsrente'] = None  # Initialize the new column

# Iterate through each row in result_df
for index, row in result_df.iterrows():
    # Find the corresponding row in rente_df based on the condition
    corresponding_row = rente_df[rente_df['Toetsinkomen'] < row['house_spendable_income']].iloc[-1]
    
    # Update the 'toetsrente' column in result_df with the value from rente_df
    result_df.at[index, 'Toetsrente'] = corresponding_row['Toetsrente']

# Now result_df has a new column 'toetsrente' with the desired values
result_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income,Toetsrente
0,75526,0,True,59122,0,95233,3.35
1,134600,1200,False,101412,0,120200,3.45
2,144798,0,True,149733,0,197999,3.45
3,150580,0,True,47582,0,166441,3.45
4,186763,0,False,181201,0,186763,3.45
...,...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811,2.85
996,111108,0,False,38953,0,111108,3.45
997,51688,0,True,47980,0,67681,3.0
998,193117,0,True,183747,0,254366,3.45


In [12]:
result_df['mortgage_year'] = result_df['house_spendable_income'] * result_df['Toetsrente'] * 0.01
result_df['mortgage_month'] = result_df['mortgage_year'] / 12
result_df

,gross_salary,alimony_amount,has_spouse,sp_gross_salary,sp_alimony_amount,house_spendable_income,Toetsrente,mortgage_year,mortgage_month
0,75526,0,True,59122,0,95233,3.35,3190.3055,265.858792
1,134600,1200,False,101412,0,120200,3.45,4146.9,345.575
2,144798,0,True,149733,0,197999,3.45,6830.9655,569.247125
3,150580,0,True,47582,0,166441,3.45,5742.2145,478.517875
4,186763,0,False,181201,0,186763,3.45,6443.3235,536.943625
...,...,...,...,...,...,...,...,...,...
995,76791,1415,False,101711,0,59811,2.85,1704.6135,142.051125
996,111108,0,False,38953,0,111108,3.45,3833.226,319.4355
997,51688,0,True,47980,0,67681,3.0,2030.43,169.2025
998,193117,0,True,183747,0,254366,3.45,8775.627,731.30225


In [ ]:
##############################################3

In [24]:
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client()

QUERY = '''
    SELECT gross_salary, gross_salary*0.30/12 * ((1 - POWER(1 + 0.0037, -360)) / 0.0037) AS MortgageValue FROM `degroup11.group11dataset.individuals` LIMIT 10
'''

# Run the query
query_job = client.query(QUERY)
rows = query_job.result()

# Get column names
columns = [field.name for field in query_job.result().schema]

# Print column names
print(" | ".join(columns))

# Print separator line
print("-" * (15 * len(columns) - 1))

# Print rows
for row in rows:
    print(" | ".join([str(row[column]) for column in columns]))


gross_salary | MortgageValue
-----------------------------
75526 | 375283.3580552769
134600 | 668817.8904515038
144798 | 719491.0319583719
150580 | 748221.381457559
186763 | 928012.1521128841
59116 | 293743.2274289086
189883 | 943515.2116835281
157499 | 782601.403627202
171667 | 853001.1946518448
168304 | 836290.6852492562


In [23]:
QUERY = ('SELECT gross_salary, gross_salary*0.30/12 * ((1 - POWER(1 + 0.0037, -360)) / 0.0037) AS MortgageValue FROM `degroup11.group11dataset.individuals` LIMIT 10')
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)


BadRequest: 400 Unrecognized name: MAX_MONTHLY_MORTGAGE at [5:9]

Location: US
Job ID: 37f83a0f-abaa-4979-b19c-610a0537fed0


In [5]:
# Perform a query.
QUERY = (
    'SELECT * FROM `degroup11.group11dataset.individuals` LIMIT 100')   # use the correct project id, etc.
query_job = client.query(QUERY)  # API request
rows = query_job.result()  # Waits for query to finish

for row in rows:
    print(row)

Row((106, 18, True, 106, 75526, False, 0, True, 9288, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((153, 18, False, 153, 134600, False, 0, True, 5314, True, 1200), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((211, 18, True, 211, 144798, True, 26010, True, 1815, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 'student_loan_amount': 6, 'has_general_loan': 7, 'general_loan_amount': 8, 'has_alimony': 9, 'alimony_amount': 10})
Row((354, 18, True, 354, 150580, True, 52758, True, 8222, False, 0), {'ID': 0, 'age': 1, 'has_spouse': 2, 'spouse_ID': 3, 'gross_salary': 4, 'has_student_loan': 5, 

In [7]:
query_job

QueryJob<project=degroup11, location=US, id=d483567c-0717-40c0-9467-7ad500b0797f>